In [1]:
%matplotlib qt
import sympy as sp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from json import dump, load
from subprocess import call
from matplotlib.animation import FuncAnimation

In [2]:
config = {
    'init' : {
        'q' : 3,
        'v' : 0
    },
    'pars' : {
        'a' : .1,
        'nu': 100,
        'l' : 1,
        'g' : 9.81
    },
    'modeling' : {
        'time' : 20,
        "dt": .001,
        "steps" : 50
    }
}

In [3]:
def set_func(config, val_type = 'double', method = 'rk45'):
    a, nu, g, l = sp.symbols('a nu g l')
    t, phi = sp.symbols('t s[0]')
    expr1 = a * nu ** 2 * sp.cos(nu * t) + g
    expr2 = sp.sin(phi) / l
    expr = -expr1 * expr2
    out = open('function.h', 'w')
    print('#include <valarray>', file=out)
    print('#define ValType '+val_type, file=out)
    print('#define METHOD "'+method+'"', file=out)
    print('std::valarray<ValType> F(ValType, std::valarray<ValType>);', file=out)
    out.close()
    out = open('function.cpp', 'w')
    print('#include <valarray>', file=out)
    print('#include <string>', file=out)
    print('#include <cmath>', file=out)
    print('#include "function.h"', file=out)
    print('using namespace std;', file=out)
    print('valarray<ValType> F(ValType t, valarray<ValType> s) {', file=out)
    print('\treturn valarray<ValType> {s[1], ', 
        str(expr.subs({
            a :config['pars']['a'],
            l :config['pars']['l'],
            g :config['pars']['g'],
            nu:config['pars']['nu'],
        })),
        '};', file=out)
    print('}', file=out)
    out.close()

In [4]:
def make_json(config):
    out = open('config.json', 'w')
    dump({
        'init' : [
            config['init']['q'],
            config['init']['v']
        ],
        'modeling' : {
            'time' : config['modeling']['time'],
            'dt': config['modeling']['dt'],
            'steps' : config['modeling']['steps']
        },
        'pars' : {
        'l' : config['pars']['l'],
        'a' : config['pars']['a'],
        'nu' : config['pars']['nu'],
        'g' : config['pars']['g']
        }
    }, out, indent=4)
    out.close()

In [5]:
def count(config):
    make_json(config)
    call('make')
    call('main.exe')
    call('make clean')

In [6]:
def load_data():
    d = pd.DataFrame(np.fromfile('output.binary', dtype=np.dtype([
    ('q', np.float64),
    ('v', np.float64)
    ])))
    return d

In [7]:
set_func(config)
count(config)
data = load_data()

In [10]:
fig, ax = plt.subplots()
fig.set_size_inches(14,7)

t = np.linspace(0, len(data) * config['modeling']['dt'], len(data))
#ax.plot(t, data.q)
ax.plot(config['pars']['l'] * np.sin(data.q), -config['pars']['l'] * np.cos(data.q) + config['pars']['a'] * np.cos(config['pars']['nu'] * config['modeling']['dt'] * np.arange(len(data))))

ax.grid(True, alpha=.3)
plt.show()

In [8]:
FPS = .1*config['modeling']['dt']

L = config['pars']['l']
H = config['pars']['l'] + config['pars']['a']

fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(autoscale_on=False, xlim=(-10*L, 10*L), ylim=(-1.02*H, 1.02*H))
ax.set_aspect('equal')
ax.grid(True, alpha=.3)

line, = ax.plot([], [], '.-', lw=2, zorder=2)
trace, = ax.plot([], [], '.-', lw=1, ms=1, zorder=1)
time_template = 'time = %.1fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)

t = np.arange(0, config['modeling']['time'], config['modeling']['dt'])
x = config['pars']['l'] * np.sin(data.q)
y =-config['pars']['l'] * np.cos(data.q) + config['pars']['a'] * np.cos(config['pars']['nu'] * config['modeling']['dt'] * np.arange(len(data)))
ax.set_xlim(min(x), max(x))
ax.set_ylim(min(min(y)-L, -config['pars']['a']), max(max(y)+L, config['pars']['a']))

def animate(i):
    thisx = [0, x[i]]
    thisy = [config['pars']['a'] * np.cos(config['pars']['nu'] * config['modeling']['dt'] * i), y[i]]

    line.set_data(thisx, thisy)
    if i > len(x):
        trace.set_data(x[0], y[0])
    else:
        trace.set_data(x[:i], y[:i])

    time_text.set_text(time_template % (i*config['modeling']['dt']))
    return line, trace, time_text

ani = FuncAnimation(fig, animate,
    frames=np.arange(0, int(config['modeling']['time'] / config['modeling']['dt'])),
    blit=True, interval=FPS)

plt.show()